In [80]:
import requests
from bs4 import BeautifulSoup
import re

def parse_text(unparsed):
    parsed = unparsed.get_text().lstrip('\n\r\n').lstrip('\n\r\n ').rstrip('\n\r\n').rstrip('\n\r\n ')
    return parsed

def extract_uuid(string):
    uuid_regex = re.compile("[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}")
    return re.search(uuid_regex, string).group(0)

def extract_year(string):
    year_regex = re.compile("[0-9]{4}")
    return re.search(year_regex, string).group(0)


In [81]:
cazoo_page = requests.get("https://www.cazoo.co.uk/cars/mercedes-benz/a-class/?colour=Grey&fuelType=Petrol")
cinch_page = requests.get("https://www.cinch.co.uk/find-vehicle?bodyType=&colour=grey&doors=&driveType=&features=&fromEngineSize=-1&fromPrice=-1&fromYear=-1&fuelType=petrol&make=Mercedes-Benz&mileage=-1&pageNumber=1&pageSize=32&seats=&selectedModel=A-Class&sortingCriteria=3&tags=&toEngineSize=-1&toPrice=-1&toYear=-1&transmissionType=&useMonthly=false&variant=")

In [98]:
cazoo_soup = BeautifulSoup(cazoo_page.content, 'html.parser')
cazoo_search_results = cazoo_soup.find_all("div", {"class" : re.compile('.*vehicle-cardstyles__Card-sc.*')})


In [99]:
#Cazoo Listings Dictionary

cazoo_listings = []



In [100]:
#Cazoo Listings

for s in cazoo_search_results:
    
    car_id = extract_uuid(s.find('a', href=True)['href'])

    details = s.findAll("div", {"class" : re.compile('.*vehicle-cardstyles__DetailWrap-sc.*')})
    
    for d in details:
        tags = d.find_next('ul').find_all('li')

        title = parse_text(d.find("p", {"class": re.compile('.*vehicle-cardstyles__Title-sc.*')}))
        description = parse_text(d.find("p", {"class" : re.compile('.*vehicle-cardstyles__DisplayVariant-sc.*')}))
        mileage = parse_text(tags[0])
        year = extract_year(parse_text(tags[1]))
        transmission = parse_text(tags[2])
        fuel = parse_text(tags[3])

    price = parse_text(s.find("p", {"data-test-id": "card-pricing-full-price-gb"}))

    listing_details = { 'car_id': car_id,
                        'car_title': title,
                        'car_decsription': description,
                        'mileage': mileage,
                        'year': year,
                        'transmission': transmission,
                        'fuel': fuel,
                        'price': price,
                        'source': 'cazoo'
                        }
    
    cazoo_listings.append(listing_details)
